# Server

In [1]:
import tekore as tk
from dotenv import load_dotenv

import server

load_dotenv()

conf = tk.config_from_environment()
cred = tk.Credentials(*conf)
spotify = tk.Spotify()
host = '127.0.0.1'
port = 5000

# Multiprocessing
app = server.SingleUseAuthServer(host, port, spotify, cred)
token = app.spawn_single_use_server()
spotify.token = token

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Jan/2023 17:11:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2023 17:11:36] "GET /login HTTP/1.1" 307 -


# Spotify API

In [2]:
spotify.token = cred.refresh(spotify.token)

In [3]:
from project.random_track import RandomTrack

rndTrack = RandomTrack(spotify, cred)
tracks = rndTrack.random_tracks()
for track in tracks:
    print(track.name)

Infinity
In Waves
Insecure
I Am Above
In the Snow
Pain Remains I: Dancing Like Flames
Fick ihn doch
Get It Crackin - Sefa Remix
Das ist alles von der Kunstfreiheit gedeckt
I Wanna Boi
Let It Go
Where Is My Mind? - Remastered
I Gotta Feeling
I'm Nuclear
Dies Irae feat. Black Prez
Flowers in your hair
Let It Go
I'm Good (Blue)
In the End
I'm Still Standing
In Da Club
From the Heart (I Hate You)
I Ain't Worried
Zukunft Pink (feat. Inéz)
In der Weihnachtsbäckerei
All I Want for Christmas Is You
Infinity
I Wanna Be Yours
Happy Xmas (War Is Over) - Remastered 2010
Ich hass dich
Snowman
I'll Be Waiting
It's Beginning to Look a Lot like Christmas
I Believe
How Do I Say Goodbye
Wonderful Christmastime - Edited Version / Remastered 2011
It's the Most Wonderful Time of the Year
It's Beginning to Look a Lot Like Christmas (with Mitchell Ayres & His Orchestra)
Let It Snow! Let It Snow! Let It Snow!
It's Beginning to Look a Lot like Christmas
Sava i Dunav
Call It Love
In Your Eyes
Thank God It's Chr

In [4]:
from project.util import Partition

artist_id_limit=50
artist_ids = [artist.id for track in tracks for artist in track.artists]
artist_id_partitions = Partition(artist_ids)
artists = [spotify.artists(ids) for ids in artist_id_partitions]
artists = [artist for partition in artists for artist in partition]
artists

[FullArtist with fields:
   external_urls = {'spotify'}
   followers = Followers(href, total)
   genres = [3 x str]
   href = 'https://api.spotify.com/v1/artists/0caJEGgVuXuSHhhrMCmlkI'
   id = '0caJEGgVuXuSHhhrMCmlkI'
   images = [3 x Image(height, url, width)]
   name = 'Sefa'
   popularity = 60
   type = 'artist'
   uri = 'spotify:artist:0caJEGgVuXuSHhhrMCmlkI',
 FullArtist with fields:
   external_urls = {'spotify'}
   followers = Followers(href, total)
   genres = [3 x str]
   href = 'https://api.spotify.com/v1/artists/6L7a6wPGpvLtTwOsMLnF1z'
   id = '6L7a6wPGpvLtTwOsMLnF1z'
   images = [3 x Image(height, url, width)]
   name = 'D-Block & S-te-Fan'
   popularity = 61
   type = 'artist'
   uri = 'spotify:artist:6L7a6wPGpvLtTwOsMLnF1z',
 FullArtist with fields:
   external_urls = {'spotify'}
   followers = Followers(href, total)
   genres = [6 x str]
   href = 'https://api.spotify.com/v1/artists/278ZYwGhdK6QTzE3MFePnP'
   id = '278ZYwGhdK6QTzE3MFePnP'
   images = [3 x Image(height, 

In [5]:
import model.track as model
analyzed_tracks = model.AnalyzedTracks(tracks, artists)

# Firebase

In [6]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.ApplicationDefault()
firebase_admin.initialize_app(cred)

In [7]:
from google.cloud.firestore_v1 import DocumentReference, Client

firestore_client: Client = firestore.client()
analyzed_tracks.upsert(firestore_client.collection("tracks"))